Copyright (c) Meta Platforms, Inc. and affiliates.
All rights reserved.

This source code is licensed under the license found in the
LICENSE file in the root directory of this source tree.

- Firt download the CMU Motion Capture Dataset from http://mocap.cs.cmu.edu/faqs.php
- Specifically download the `allc3d_0.zip` file linked in that page.
- Create directories `cmu_dataset/allc3d_0/` and move the `allc3d_0.zip` file therein. Then unzip it.
- You should then have a directory `cmu_dataset/allc3d_0/subjects/` which is where the below code loads the data from.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set_palette("husl")
import c3d
from pathlib import Path

from utils import load_activity, load_all_activities, joint_angles, estimate_plot, to_ndarray, parzen_plot, standardize
from parzen import ParzenWindowsGaussian
import torch

In [ ]:
# path to the dataset here. I put all the files under cmu_dataset/
BASE_DIR = Path('cmu_dataset/')

datasets_df = pd.DataFrame({'path': list(BASE_DIR.glob('allc3d_0/subjects/*/*.c3d'))})
datasets_df['Subject'] = datasets_df['path'].map(lambda x: x.parent.stem)
datasets_df['Activity'] = datasets_df['path'].map(lambda x: x.stem.split('_')[-1].lower())

In [ ]:
datasets_df

In [ ]:
# hf = open(datasets_df['path'][3], 'rb')
# reader = c3d.Reader(hf)

In [ ]:
all_act = load_all_activities(datasets_df)

In [ ]:
all_act

In [ ]:
# drop point labels starting with * followed by numbers because we don't know what it means
all_df = all_act[all_act['point_label'].str.startswith('*')==False]

In [ ]:
all_df = all_act.copy()

In [ ]:
agg_df = all_df.pivot_table(columns="point_label", values=["x", "y", "z", "frame", "cam", "err"], index="time")

In [ ]:
agg_df

In [ ]:
flat_index = [ b+'_'+a for a,b in agg_df.columns.to_flat_index()]

In [ ]:
flat_index

In [ ]:
df_grouped = agg_df.copy()
df_grouped.columns = flat_index

In [ ]:
df_grouped

In [ ]:
shoulders = joint_angles(df_grouped, 'RSHO', 'LSHO')

In [ ]:
left_elbow = joint_angles(df_grouped, 'LELB', 'LUPA', 'LWRA')

In [ ]:
straight = np.vstack((shoulders, left_elbow)).T

In [ ]:
straight, _, _ = standardize(straight)

In [ ]:
straight = torch.tensor(straight)

In [ ]:
straight.device

In [ ]:
f, _, _ = parzen_plot(straight, xlabel="angle(shoulder)", ylabel="angle(left elbow)", levels=20, std=False, title="", arrows=False, scale_units='height', sigma=0.15, n_points=100, cbar_label="magnitude of density gradient", fontsize=16)
plt.savefig('mocap.pdf')